In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import glob
from pathlib import Path
import rasterio
from shapely.geometry import box

In [ ]:
pwd

In [ ]:
insar_path = Path('../asc_lower')

In [ ]:
dem_path = [Path(x) for x in insar_path.glob('./*/*dem_clip.tif')][0]

with rasterio.open(dem_path) as meta:
    crs = meta.crs
    transform = meta.transform
    bounds = meta.bounds
    dem = meta.read(1)

bboxras = box(*list(bounds))

In [ ]:
ls_path = Path('mapped_landslides')

ls = gpd.read_file(ls_path / 'large_landslidesV2.shp').to_crs(str(crs))

bbox = box(*ls.total_bounds)
bbox10 = bbox.buffer(10000)

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(*bboxras.exterior.xy)
ax.plot(*bbox10.exterior.xy)
ax.plot(*bbox.exterior.xy)

In [ ]:
def coord2pixels(bounds,transform,shape):
    xstep, _, xstart, _, ystep, ystart, _, _, _ = [int(x) for x in transform]
    minx, miny, maxx, maxy = [int(x) for x in bounds]
    
    x0 = int((minx - xstart) / xstep)
    x1 = int((maxx - xstart) / xstep)
    
    y0 = int((maxy - ystart) / ystep)
    y1 = int((miny - ystart) / ystep)
    
    y0, x0 = [max(a,0) for a in (y0, x0)]
    y1, x1 = [min(a,b) for a,b in zip((y1, x1),shape)]
    
    ratio = int(((y1-y0) * (x1-x0)) / (shape[0]*shape[1]) * 100)
    print(f'Window is {ratio}% the size of the origional image')

    return [[y0,y1],[x0,x1]]

In [ ]:
window = coord2pixels(bbox10.bounds,transform,dem.shape)
print(f'{window[0][0]}:{window[0][1]},{window[1][0]}:{window[1][1]}')

In [ ]:
plt.imshow(dem,cmap='gray')

In [ ]:
slice_obj = slice(window[0][0],window[0][1]),slice(window[1][0],window[1][1])
plt.imshow(dem[slice_obj],cmap='gray')

In [ ]:
boxes = {'asc_upper':'4006:6555,658:5440',
        'asc_lower':'0:2394,0:4120',
        'desc_upper':'4988:6401,1584:6366',
        'desc_lower':'744:3316,2742:7481'}